## Molecular Dynamics (MD) Integration##
In our last exercise, we used a very simple "hard wall" description of the interaction of our particle with the walls of the box. A real particle, of course, interacts with other objects via various *forces* that describe how strongly the objects attract or repel each other. In this exercise, we'll treat these forces a bit more realistically. To do that, we'll need some familiarity with the concept of a *molecular dynamics integrator* -- a computer algorithm that numerically integrates Newton's laws for classical particles. 

Newton's second law $${\bf F} = M {\bf a}$$ states that the acceleration ${\bf a} = \ddot {\bf x}$ of a particle is proportional to the force ${\bf F}$ acting on it and inversely proportional to its mass $M$. Typically the force ${\bf F}$ is derived as the (negative) gradient of an interaction potential, i.e., 
\begin{align}
{\bf F}({\bf x}) &= - \nabla V({\bf x})
\end{align}
that describes the interaction energy of the particle with other objects (e.g., other particles or the walls of a box). In principle, if we know the functional form ${\bf F}({\bf x})$ for how the force $\bf F$ varies with the position ${\bf x}$ of the particle, we can integrate this equation to calculate the position trajectory ${\bf x}(t)$. In practice, analytical solutions are not possible, and the equations must be solved numerically. An MD integrator does exactly this. 


## Euler Integration ##
The simplest MD integrator we could use is an algorithm known as Euler's method. Suppose we are given the values ${\bf x}(0)$ and ${\bf v}(0)$ for the position and velocity of the particle at time $t = 0$, and we know the functional form ${\bf F}({\bf x})$ for the force on the particle. Euler's method then approximates the position at a very small time step time $t = \Delta t$ away from $t = 0$ as 
$${\bf x}(\Delta t) \approx {\bf x}(0) + {\bf v}(0) \cdot \Delta t . $$
So the distance the particle moves in the time interval $\Delta t$ is just determined by its initial velocity at $t = 0$. With this updated position ${\bf x}(\Delta t)$, we can now calculate the force on the particle $ {\bf F}(\bf{x(\Delta t)})$ and, consequently, the acceleration 
$${\bf a}(\Delta t) = \frac{{\bf F}({\bf x}(\Delta t))}{M}.$$
Finally, the velocity ${\bf v}(\Delta t)$ is estimated from the acceleration: since $\dot {\bf v}(t) = {\bf a}(t)$,
$${\bf v}(\Delta t) \approx {\bf v}(0) + {\bf a}(\Delta t) \cdot \Delta t.$$

This completes one step in Euler integration. With the updated parameters ${\bf x}(\Delta t)$ and ${\bf v}(\Delta t)$, we can now repeat the calculation to estimate ${\bf x}(2 \Delta t)$, ${\bf a}(2 \Delta t)$, and ${\bf v}(2 \Delta t)$ -- another step. Iterating the process over and over again, we obtain an estimated trajectory for ${\bf x}(n \Delta t)$ as 
\begin{align}
{\bf x}(n \Delta t) &\approx {\bf x}((n-1) \Delta t) + {\bf v}(n \Delta t) \Delta t\\
{\bf a}(n \Delta t) &\approx \frac{{\bf F}({\bf x}(n \Delta t) )}{M}\\
{\bf v}(n \Delta t) &\approx {\bf v}((n-1) \Delta t) + {\bf a}(n \Delta t) \Delta t\\
\end{align}
Thus, in Euler intergration the position and velocity at each time step are estimated solely from the position and velocity at the previous time step. If the time step $\Delta t$ is chosen to be very small (relative to the time it takes for the force on the particle to change substantially), Euler's method can provide a very accurate approximation to the true Newtonian dynamics. 

## Velocity Verlet Integration ##

Unfortunately, for practical purposes, Euler's method is not very useful: the time step $\Delta t$ must be chosen *so small* that MD simulations for real systems become intractible. (Note that the number of computational steps in the simulation is proportional to the length of the simulation $t_\text{sim}$ *divided by* the time step $\Delta t$. So, for example, a 100 ns simulation with a 1 fs time step represents $10^{-7}/10^{-15} = 10^8$ simulation steps! For this reason, a variety of other integrators have been devised that allow for much larger time steps than the Euler method. Mathematically, these algorithms correspond to calculating a *second order* Taylor expansion for the position $x(t)$ at each time step. (We'll talk more about Taylor expansions in the coming weeks.) 

The algorithm we'll use in this class is *velocity Verlet* integration. The basic steps in velocity Verley integration are
1. Calculate ${\bf x}_n = {\bf x}_{n-1} + {\bf v}_{n-1} \Delta t + \frac{1}{2}{\bf a}_{n-1} \Delta t^2$.
2. Calculate ${\bf a}_n$ from the force function ${\bf F}({\bf x})$ and the new position ${\bf x}_n$.
3. Calculate ${\bf v}_n = {\bf v}_{n-1} + \frac{{\bf a}_n + {\bf a}_{n-1}}{2} \Delta t$ using the *average velocity* from the two most recent time steps.

The velocity Verlet algorithm is only slightly more complicated to code than the original Euler algorithm, but it affords a drastic stabilization of the dynamics. Similar methods include the Verley algorithm (in which the velocity is never explicitly calculated) and Leap Frog integration (where the velocity is calculated at the mid-point between position time-steps). 


## Particle in a Soft-Wall Box ##
To see how MD integration works in practice, we'll repeat our particle-in-a-box MD simulation from last week with a particle contained in a "soft wall" box where the particle interacts with each wall via a potential of the form
\begin{align}
V(r) = \varepsilon_o \frac{R_o^4}{r^4}
\end{align}
where $r$ is the (shortest) distance between the particle and the wall, $\varepsilon_o$ is an interaction energy parameter, and $R_o$ controls how closely the particle must approach the wall before it starts to be repelled. For example, the potential for the interaction of the particle with the left wall in our simulation box (running along the line ${\bf x} = 0$) is 
\begin{align}
V^{(\text{left})}({\bf x}) = \varepsilon_o \frac{R_o^4}{x^4}.
\end{align}
From this potential, we calculate the force exerted on the particle by the left wall as
\begin{align}
{\bf F}^{(\text{left})}({\bf x}) = -\nabla V^{(\text{left})}({\bf x}) = 4 \varepsilon_o \frac{R_o^4}{x^5}.
\end{align}
Similarly, the potential for the right wall is given by 
\begin{align}
V^{(\text{right})}({\bf x}) = \varepsilon_o \frac{R_o^4}{(L-x)^4},
\end{align}
leading to a force
\begin{align}
{\bf F}^{(\text{right})}({\bf x}) = - 4 \varepsilon_o \frac{R_o^4}{(L-x)^5}.
\end{align}
Forces for interaction with the top and bottom of the box take similar forms but with $y$ in place of $x$. 

In implementing the algorithm, we can keep our code much cleaner by defining *functions* that calculate specific quantities -- e.g. the acceleration for a given particle position -- and then just calling them as we iterate through the simulation. In python, functions are defined by a `def` block. In the code below, we start by defining two functions, `calc_accel` (to calculate the acceleration on our particle at a given position) and `vv_step` that updates positions and velocities via a single step of the velocity Verlet algorithm. Notice here that although both functions take specific arguments (e.g., `calc_accel` takes the positions `x` and `y`), they can access other variables (e.g., L, dt, M, etc.) that are defined earlier in the code but not passed explicitly to the function. This can be very convenient, but it also means you'll have to pay attention to which variables have been defined! Along with introducing these two new functions, we define two new parameters, `epso` and `Ro`, corresponding to the particle/wall interaction parameters $\varepsilon_o$ and $R_o$. 

Otherwise, the code is similar to what we ran last week, although we've updated all units to follow the cgs (centimeter-gram-second) system and made the box much smaller (only 10 nm long). We've also moved our plotting code inside of an `if(n%1000==0)` block that results in the plot updating only every 1000 time-steps. (Otherwise, the program spends most of its time updating the plot, and the simulation runs very slowly.) Once you've reviewed the code and think you understand roughly how it works, run it by pressing <shift+enter>.

In [ ]:
import math
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time

def calc_accel(x,y):
    ax = 4*epso*(Ro**4)/(M*x**5) - 4*epso*(Ro**4)/(M*(L-x)**5)
    ay = 4*epso*(Ro**4)/(M*y**5) - 4*epso*(Ro**4)/(M*(L-y)**5)
    return ax,ay
    
def vv_step(x,y,vx,vy,ax,ay):
    xnew = x + vx*dt + 0.5*ax*dt*dt
    ynew = y + vy*dt + 0.5*ay*dt*dt
    axnew,aynew = calc_accel(xnew,ynew)
    vxnew = vx + 0.5*(ax + axnew)*dt
    vynew = vy + 0.5*(ay + aynew)*dt
    return xnew,ynew,vxnew,vynew,axnew,aynew

epso = 190.0*(1.38064852e-23)*(1e+3)*(1e+4)
Ro = 5.0e-8

tmax=50e-12         # Total simulation time in seconds
dt=1e-15            # Time-step in seconds
Nsteps=int(round(tmax/dt))     
M=4*(1.66054e-24)   # Mass in g
L=1e-6              # Box length in cm
V=1e+5              # Velocity (magnitude) in cm/s

x=0.5*L                               # initial x-coordinate (meters)
y=0.5*L                               # initial y-coordinate (meters)
vx = 1                                # initial x-component of velocity (arbitrary units)
vy = 2                                # initial y-component of velocity (arbitrary units)
NormFac = math.sqrt(vx*vx + vy*vy)    # This square-root command is why we needed to import the math module
vx = V*vx/NormFac                     # Normalize so that total velocity is V
vy = V*vy/NormFac   

ax,ay = calc_accel(x,y)

for n in range(0,Nsteps):
    x,y,vx,vy,ax,ay = vv_step(x,y,vx,vy,ax,ay)
    if(n%1000==0):
        plt.clf()
        plt.text(0.1*L,0.1*L,'t = '+str(round(n*dt*1e+12))+' ps')
        plt.xlim([0,L])
        plt.ylim([0,L])
        plt.plot(x,y,'ko')
        display.display(plt.gcf())
        display.clear_output(wait=True)

Note that the particle now begins gradually to be repelled by the wall before actually reaching it. This is what we mean by a ''soft wall'' simulation. 


## Multi-Particle Simulation ##
Next, let's extend our simulation to include multiple particles. This is easiest if we use the NUMerical PYthon (NumPy) package which allows for vector and matrix manipulations with a syntax very similar to Matlab. Using Numpy, the only real change we need to make to our code is to change the scalar variables `x`, `y`, `vx`, `vy`, `ax`, and `ay` to vector variables which we'll denote `X`, `Y`, etc. Numpy's ability to perform operations like addition, subtraction, and multiplication to entire arrays will allow us to leave our functions `calc_accel` and `vv_step` completely unchanged. 

First we import the numpy package with the command `import numpy as np` at the top of the cell block. Then in the comment heading "Key Changes" we initialize vector variables for position, velocity, and acceleration. The numpy function `np.random.random()` returns a vector of random numbers between 0 and 1. Similarly, `np.random.normal()` returns a vector of Gaussian random numbers. 

In [ ]:
import math
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time

import numpy as np

def calc_accel(x,y):
    ax = 4*epso*(Ro**4)/(M*x**5) - 4*epso*(Ro**4)/(M*(L-x)**5)
    ay = 4*epso*(Ro**4)/(M*y**5) - 4*epso*(Ro**4)/(M*(L-y)**5)
    return ax,ay
    
def vv_step(x,y,vx,vy,ax,ay):
    xnew = x + vx*dt + 0.5*ax*dt*dt
    ynew = y + vy*dt + 0.5*ay*dt*dt
    axnew,aynew = calc_accel(xnew,ynew)
    vxnew = vx + 0.5*(ax + axnew)*dt
    vynew = vy + 0.5*(ay + aynew)*dt
    return xnew,ynew,vxnew,vynew,axnew,aynew

epso = 190.0*(1.38064852e-23)*(1e+3)*(1e+4)
Ro = 5.0e-8

tmax=50e-12         # Total simulation time in seconds
dt=1e-15            # Time-step in seconds
Nsteps=int(round(tmax/dt))
M=4*(1.66054e-24)   # Mass in g
L=1e-6              # Box length in cm
V=1e+5              # Velocity (magnitude) in cm/s


##############################################
################# Key Changes ################

# Define the number of particles
Npart = 5

# Generate a vector of random particle positions
X = 0.1*L + np.random.random((Npart))*0.8*L
Y = 0.1*L + np.random.random((Npart))*0.8*L

V = 79000
# Generate a vector of random velocities
VX = np.random.normal(0, V, (Npart))
VY = np.random.normal(0, V, (Npart))

# Calculate accelerations at initial positions
AX,AY = calc_accel(X,Y)

##############################################


for n in range(0,Nsteps):
    X,Y,VX,VY,AX,AY = vv_step(X,Y,VX,VY,AX,AY)
    if(n%500==0):
        plt.clf()
        plt.text(0.1*L,0.1*L,'t = '+str(round(n*dt*1e+12))+' ps')
        plt.xlim([0,L])
        plt.ylim([0,L])
        plt.plot(X,Y,'ko')
        display.display(plt.gcf())
        display.clear_output(wait=True)

## Inter-particle Interactions ##

Now that we have multiple particles in our box, let's make things more interesting by letting them interact with each other. Specifically, let's introduce two interaction potentials for the interaction between particles $m$ and $n$: an electrostatic potential 
\begin{align}
V_{mn}^{\text{ES}}\left({\bf x}^{(m)},{\bf x}^{(n)} \right) = \frac{Q_m Q_n}{ \left \| {\bf x}^{(m)} - {\bf x}^{(n)} \right \|}
\end{align}
and a Lennard-Jones or 6-12 potential
\begin{align}
V_{mn}^{\text{LJ}}\left({\bf x}^{(m)},{\bf x}^{(n)} \right) &= \varepsilon_o \left ( \frac{R_o^{12}}{\left \| {\bf x}^{(m)} - {\bf x}^{(n)} \right \|^{12}} - 2 \frac{R_o^6}{\left \| {\bf x}^{(m)} - {\bf x}^{(n)} \right \|^6} \right ) .
\end{align}
The electrostatic potential is exactly the Coulomb potential, which we saw in lecture is relevant in the near-field regime, where particle velocities and distances are small relative to the speed of light. The Lennard-Jones potential is designed to mimic non-bonding interactions such as Van Der Waal's forces between atoms. From these two potentials, the $x$-component of the force on the $n^{\text{th}}$particle may be calculated as
\begin{align}
F_x^{(n)} &= - \sum_{m \neq n} \frac{\partial V_{mn}^{(\text{ES})}}{\partial x^{(n)}} - \sum_{m \neq n} \frac{\partial V_{mn}^{(\text{LJ})}}{\partial x^{(n)}} ,
\end{align}
and similarly for the $y$ and $z$ components. For example, the electrostatic contribution to the force is 
\begin{align}
- \sum_{m \neq n} \frac{\partial V_{mn}^{(\text{ES})}}{\partial x^{(n)}} = \sum_{m \neq n} Q_m Q_n \frac{x^{m} - x_n}{ \left \| {\bf x}^{(n)} - {\bf x}^{(m)} \right \|^3} .
\end{align}

In the code below, we introduce the two potentials in two sections labeled "Key Changes". Note that in Python (as in C-based languages) the syntax ``a += b`` is shorthand for ``a = a + b``. In the new code, the user can choose separately the number ``Nneg`` of negatively-charged particles and the number ``Npos`` of positively-charged particles. 

In [ ]:
import math
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time

import numpy as np

def calc_accel(x,y):
    ax = 4*epso*(Ro**4)/(M*x**5) - 4*epso*(Ro**4)/(M*(L-x)**5)
    ay = 4*epso*(Ro**4)/(M*y**5) - 4*epso*(Ro**4)/(M*(L-y)**5)
    ############### Key Changes ###############
    for p1 in range(0, Npart):
        for p2 in range(0,Npart):
            if p1!=p2:
                rX = x[p1] - x[p2]
                rY = y[p1] - y[p2]
                r = math.sqrt(rX*rX + rY*rY)
                ax[p1] += Q[p1]*Q[p2]*rX/(M*r**3)
                ay[p1] += Q[p1]*Q[p2]*rY/(M*r**3)
                ax[p1] += 12.0*epso*rX*(Ro**12)/(M*r**14) - 12.0*epso*rX*(Ro**6)/(M*r**8)
                ay[p1] += 12.0*epso*rY*(Ro**12)/(M*r**14) - 12.0*epso*rY*(Ro**6)/(M*r**8)
    ############################################
    return ax,ay
    
def vv_step(x,y,vx,vy,ax,ay):
    xnew = x + vx*dt + 0.5*ax*dt*dt
    ynew = y + vy*dt + 0.5*ay*dt*dt
    axnew,aynew = calc_accel(xnew,ynew)
    vxnew = vx + 0.5*(ax + axnew)*dt
    vynew = vy + 0.5*(ay + aynew)*dt
    return xnew,ynew,vxnew,vynew,axnew,aynew

def update_plot():
    plt.clf()
    plt.text(0.1*L,0.1*L,'t = '+str(round(n*dt*1e+12))+' ps')
    plt.xlim([0,L])
    plt.ylim([0,L])
    plt.plot(X[0:Npos],Y[0:Npos],'ro')
    plt.plot(X[Npos:],Y[Npos:],'bo')
    plt.xticks([]) 
    plt.yticks([]) 
    display.display(plt.gcf())
    display.clear_output(wait=True)

epso = 190.0*(1.38064852e-23)*(1e+3)*(1e+4)
Ro = 5.0e-8

tmax=10e-12         # Total simulation time in seconds
dt=1e-15            # Time-step in seconds
Nsteps=int(round(tmax/dt))
M=4*(1.66054e-24)   # Mass in g
L=1e-6              # Box length in cm
V=1e+5              # Velocity (magnitude) in cm/s


##############################################
################# Key Changes ################

# Number of positive particles
Npos = 5

# Number of negative particles
Nneg = 5

# Total number of particles
Npart = Npos+Nneg

# Particle charges
Qo = 4.803e-10         # Elementary charge in statCoulombs
Q = np.zeros((Npart))  # Empty vector for particle charges
Q[0:Npos] = 0.5*Qo    # First Npos particles are positive
Q[Npos:] = -0.5*Qo     # Last Nneg particles are negative

##############################################


# Generate a vector of random particle positions
X = 0.1*L + np.random.random((Npart))*0.8*L
Y = 0.1*L + np.random.random((Npart))*0.8*L

V = 0
# Generate a vector of random velocities
VX = np.random.normal(0, V, (Npart))
VY = np.random.normal(0, V, (Npart))

# Calculate accelerations at initial positions
AX,AY = calc_accel(X,Y)

for n in range(0,Nsteps):
    X,Y,VX,VY,AX,AY = vv_step(X,Y,VX,VY,AX,AY)
    if(n%50==0):
        update_plot()

## Field-Particle Interactions ##
Finally, let's let the particles interact with an external electric field. We'll assume the field has a Gaussian profile
\begin{align}
{\bf E}(t) = {\bf E}^{(\text{max})} \cos\left( 2\pi \nu (t-t_o) \right) e^{-\frac{(t-t_o)^2}{2\sigma^2}}
\end{align}
*independent* of particle position. This assumption is justified when the wavelength of the radiation is long relative to the box size so that the field oscillates in sync across the entire box volume. The vector ${\bf E}^{(\text{max})}$ defines the direction in which the field is polarized and its maximum value. The frequency $\nu$ indicates how rapidly the field oscillates in time, and $\sigma$ represents the pulse length. According to the Lorentz force law, a particle with charge $q$ feels a force of $q {\bf E}(t)$. 

In the code below, the function ``gauss_pulse()`` calculates the $y$-component of the electric field from a Gaussian pulse polarized along the $y$ axis. The function ``calc_accel()`` is updated to include the Lorentz force acceleration on each particle, and the two plotting functions are updated to plot the electric field in a smaller window adjacent to the simulation box. Note that the electric field is specified in Gaussian (cgs) units of statV/cm. The box size has also been increased to 20 nm for purposes of visualization and stability. 

In [ ]:
import math
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time

import numpy as np

def calc_accel(x,y,efield):
    ax = 4*epso*(Ro**4)/(M*x**5) - 4*epso*(Ro**4)/(M*(L-x)**5)
    ay = 4*epso*(Ro**4)/(M*y**5) - 4*epso*(Ro**4)/(M*(L-y)**5)
    for p1 in range(0, Npart):
        for p2 in range(0,Npart):
            if p1!=p2:
                rX = x[p1] - x[p2]
                rY = y[p1] - y[p2]
                r = math.sqrt(rX*rX + rY*rY)
                ax[p1] += Q[p1]*Q[p2]*rX/(M*r**3)
                ay[p1] += Q[p1]*Q[p2]*rY/(M*r**3)
                ax[p1] += 12.0*epso*rX*(Ro**12)/(M*r**14) - 12.0*epso*rX*(Ro**6)/(M*r**8)
                ay[p1] += 12.0*epso*rY*(Ro**12)/(M*r**14) - 12.0*epso*rY*(Ro**6)/(M*r**8)
    ay += Q*efield/M
    return ax,ay
    
def vv_step(x,y,vx,vy,ax,ay,efield):
    xnew = x + vx*dt + 0.5*ax*dt*dt
    ynew = y + vy*dt + 0.5*ay*dt*dt
    axnew,aynew = calc_accel(xnew,ynew,efield)
    vxnew = vx + 0.5*(ax + axnew)*dt
    vynew = vy + 0.5*(ay + aynew)*dt
    return xnew,ynew,vxnew,vynew,axnew,aynew

def init_plot():
    fig = plt.figure(1)
    ax1 = plt.clf()
    txt = plt.text(0.1*L,0.1*L,'t = '+str(round(0))+' fs')
    negLine, = plt.plot(X[0:Npos],Y[0:Npos],'bo')
    posLine, = plt.plot(X[Npos:],Y[Npos:],'ro')
    plt.xlim([0,L])
    plt.ylim([0,L])
    plt.xticks([]) 
    plt.yticks([]) 
    
    ax2 = plt.axes([1.0,0.575,0.3,0.3])
    plt.xlabel('$t$')
    plt.ylabel('$E(t)$')
    field_line, = plt.plot(taxis,Efield)
    plt.xticks([])
    plt.yticks([])
    plt.ylim([0,1])
    plt.ylim([0,dt*Nsteps])
    return fig,ax1,ax2,negLine,posLine,txt,field_line

def update_plot(n):
    negPts.set_ydata(Y[0:Npos])
    negPts.set_xdata(X[0:Npos])
    posPts.set_ydata(Y[Npos:])
    posPts.set_xdata(X[Npos:])
    txt.set_text('t = '+str(round(n*dt*1e+15))+' fs')
    
    plt.sca(ax2)
    #plt.plot(taxis[0:n]/Emax, 'k.')
    field_line.set_xdata(taxis[0:n])
    field_line.set_ydata(Efield[0:n]/Emax)
    plt.ylim([-1,1])
    #plt.xlim([0,dt*Nsteps])
    
    fig.canvas.draw()
    display.display(plt.gcf())
    display.clear_output(wait=True)
    
def gauss_pulse(t):
    return Emax*np.cos(2.0*math.pi*(t-to)*nu)*np.exp(-((t-to)**2)/(2.0*sigma*sigma))

epso = 190.0*(1.38064852e-23)*(1e+3)*(1e+4)
Ro = 5.0e-8

#################### BREAK ###########################

# SIMULATION PARAMETERS #
Npos = 5
Nneg = 5
L = 2e-6
to = 500e-15
sigma = 100e-15
nu = 2e+12
Emax = 100e+4   # Maximum electric field in statV/cm

tmax=1e-12      # Total simulation time in seconds
dt=0.25e-15     # Time-step in seconds
Nsteps=int(round(tmax/dt))
M=4*(1.66054e-24)   # Mass in g
Qo = 4.803e-10         # Elementary charge in statCoulombs
taxis = np.arange(0,tmax,dt)   # Time axis (array of time steps)


# Set Particle charges
Npart = Npos+Nneg
Q = np.zeros((Npart))  # Empty vector for particle charges
Q[0:Npos] = +Qo    # First Npos particles are positive
Q[Npos:] = -Qo     # Last Nneg particles are negative

# Generate a vector of random particle positions
X = 0.1*L + np.random.random((Npart))*0.8*L
Y = 0.1*L + np.random.random((Npart))*0.8*L

# Set initial velocities to zero 
VX = np.zeros((Npart))
VY = np.zeros((Npart))

# Generate the pulse profile
Efield = gauss_pulse(taxis)

# Calculate accelerations at initial positions
AX,AY = calc_accel(X,Y,Efield[0])

fig,ax1,ax2,negPts,posPts,txt,field_line = init_plot()
for n in range(0,Nsteps):
    X,Y,VX,VY,AX,AY = vv_step(X,Y,VX,VY,AX,AY,Efield[n])
    if(n%50==0):
        update_plot(n)
        

# Homework#

## Compute ##

For your homework, you'll run this simulation code with a variety of different starting positions and parameter values. I'll outline the changes for you to make (usually with specific commands). Your job is to copy and paste into new files, modify the correct parameters, run the simulations, and answer questions based on your observations. 

To begin, make a new Jupyter Notebook and paste all the code from the last cell **above** the ``####### BREAK ########`` line. Execute this code by pressing <ctrl+enter>. All code in this cell will now be "initialized" and accessible to any other code you write below it. 

Now, for each of the tasks numbered below, copy and paste the code **below** the ``####### BREAK ########`` line (in the cell above this text) into a new cell. **You should have a separate cell for each of the four tasks.** Add a comment on the first line of this cell indicating which task it corresonds to (1, 2, 3, or 4). Edit the code/parameters according to the instructions for that task, run the simulation, and answer the questions based on what you see. Insert your answers in a markdown cell in the Jupyter Notebook **directly below the corresponding simulation code**.

1. Run a simulation with 5 positive particles and 5 negative particles. Initially the particle positions are random; positive and negative particle positions are independent of each other. Does the pulse tend to create correlations between the particle positions? (E.g., do positive and negative particles move in the same or different directions?) Why? Does the correlation last after the pulse is gone? (E.g., in the final frame, do the particle positions look random?) 


2. Set the box length (L) to 1 nm (note that simulation units are cm). Run a simulation with a single positive particle with position (x,y) = (0.5*L, 0.5*L) and zero velocity. Inside the ``if(n%50==0):`` block, add the line ``plt.plot(n*dt,Y[0]/L-0.5,'r.')`` to plot the change in $y$-coordinate of the particle (normalized to the box length) alongside the electric field. Set the pulse frequency $\nu$ to 10 THz (note: program units are in Hz = 1/s). Is the change in position of the particle in sync or out of sync with the field? Why?


3. Repeat the single-particle simulation with a pulse frequency of $\nu = 5$ THz. At the *end* of the simulation, is the particle more perturbed by the $\nu = 10$ THz field or the $\nu = 5$ THz field? Why?


4. This last task will require you to run a much longer simulation, so we're going to delete most of the plotting functions (otherwise the simulation updates very slowly). 

    (a) Reset all parameters to their initial values. Next, delete the initial call to ``init_plot()``. 
    
    (b) Replace the call to ``update_plot()`` with the command ``print(100*n/Nsteps)``; this will print to the screen the percentage completion of the simulation (much faster than drawing the entire figure). 
    
    (c) Change ``if(n%50==0):`` to ``if(n%5000==0):``; this will cause the ``print()`` statement to be evaluated on every 5000$^{\text{th}}$ step instead of every 50$^{\text{th}}$ step. 
    
    (d) Before entering the ``for()`` loop, create an empty NumPy vector of length Nsteps using the command ``MuY = np.zeros((Nsteps))``. Then at each step of the simulation (inside the ``for()`` loop), set the contents of ``MuY[n]`` equal to the $y$-component of the dipole moment using the command ``MuY[n] = np.sum(Y*Q)``. (Recall that the dipole moment is just the sum over the product of each particle's charge and position). 
    
    (e) Finally, *after* the ``for()`` loop (not inside!) plot both the electric field (``plt.plot(taxis, Efield/Emax, 'b')``) and the MuY vector (``plt.plot(taxis, MuY/np.max(MuY), 'r')``), both normalized to have a maximum value of 1. Run your code to make sure it executes successfully.
    
    With these adjustments made to the code, increase the box size to 50 nm, the number of particles to 10 positive and 10 negative, and the total simulation length to 10 ps. Decrease the maximum field strength to $3\cdot 10^5$ statV/cm Run the code. This will take a few minutes! If you run into errors, you may want to run with a shorter simulation time until the bugs are worked fixed. 
    
    Once the code executes, you should have a plot of the applied electric field (blue line) and the *induced dipole moment* (red). Is the maximum value of the dipole moment in sync or out of sync with the polarization? Why might this be physically? After the pulse is gone, does the induced dipole moment remain (roughly) static or oscillate? Why? Does it appear to decay with time? Why? 



## Calculate ##

The photosystem II antenna complex (the chlorophyll/protein engine of oxygenic photosynthesis) measures approximately 20 nm end-to-end. How long does it take light to propagate from one end of the complex to the other? Given that most vibrational motion occurs on timescales of 10 fs or greater, is the near-field approximation reasonable for simulating the atomistic dynamics of such a complex? What about for simulating the electronic dynamics (e.g., energy transfer between different chlorophyll molecules), given that electronic dynamics in the visible wavelength range occur on timescales of 1 fs or greater? 


## Contemplate ##

In our simulation, the variable ``Efield`` represents an "external field" applied to the sample from, e.g., a focused laser beam. In reality, of course, the electric field in the box is much more complicated, since each charged particle produces its own local field. Why did we not include this in calculating the Lorentz force on each particle? Under what conditions is this treatment appropriate? 


# Extra Credit #

Consider a Gaussian pulse propagating along the $z$ axis with an electric field
\begin{align}
{\boldsymbol E}({\boldsymbol x}, t) ={\boldsymbol E}^{(\text{max})} e^{- \frac{x^2+y^2}{2 \sigma_x^2}} e^{- \frac{\left (t - \frac{n z}{c} \right)^2 }{ 2 \sigma_t^2} } \cos \omega \left( \frac{nz}{c} - t \right ) .
\end{align}
If $\left \| {\boldsymbol E}^{(\text{max})} \right \| = 1 \cdot 10^4 {\text{statV/cm}}$, $\omega = 2\pi c \cdot 1650$ cm$^{-1}$, $\sigma_t = 20 $ fs, and $\sigma_x = 25 \mu$m, what is the pulse energy? Express the answer in both cgs and SI units. 